# Bsky integration

In [ ]:
import { Agent } from "@atproto/api";

const agent = new Agent("https://public.api.bsky.app/xrpc");
await agent.getProfile({ actor: "did:plc:ubdeopbbkbgedccgbum7dhsh" });

// CDN for the image
// https://github.com/notjuliet/pdsls/blob/c86372402cc5cb78c72277938adc2912b8100a85/src/components/json.tsx#L114-L129

XRPCResponse {
  data: {
    did: "did:plc:ubdeopbbkbgedccgbum7dhsh",
    handle: "callmephilip.com",
    displayName: "Philip Nuzhnyi",
    avatar: "https://cdn.bsky.app/img/avatar/plain/did:plc:ubdeopbbkbgedccgbum7dhsh/bafkreieucw6agsbrrjrmjubywrrzqqpyhdbnd4trlj63qvzwm4ildgtmsq@jpeg",
    associated: { lists: 1, feedgens: 0, starterPacks: 0, labeler: false },
    labels: [],
    createdAt: "2024-11-29T15:33:26.546Z",
    description: "🧜‍♂️ Surftware engineer.\n" +
      "\n" +
      "CODE: https://github.com/callmephilip\n" +
      "MD: https://callmephilip.com/",
    indexedAt: "2024-12-29T17:12:41.342Z",
    banner: "https://cdn.bsky.app/img/banner/plain/did:plc:ubdeopbbkbgedccgbum7dhsh/bafkreibw6qn7obhzn2fxb76v7o5tltwpdgq45wsypdzkp4ujrlwjdprlc4@jpeg",
    followersCount: 85,
    followsCount: 116,
    postsCount: 166
  },
  headers: {
    "access-control-allow-origin": "*",
    "atproto-content-labelers": "did:plc:ar7c4by46qjdydhdevvrndac;redact",
    "cache-control": "public, max